<a href="https://colab.research.google.com/github/EloneSampaio/AHUs/blob/master/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv

# Função para salvar as alocações no formato desejado
def salvar_alocacoes_csv(alocacoes, pontos_avaliacao, participantes, arquivo):
    with open(arquivo, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Ponto de Avaliação", "Cod Participante", "Nome do Participante"])

        for ponto_id, alocados in alocacoes.items():
            ponto_nome = next(p["nome"] for p in pontos_avaliacao if p["id"] == ponto_id)
            for participante_id in alocados:
                participante = next(p for p in participantes if p["id"] == participante_id)
                writer.writerow([ponto_nome, participante["id"], participante["nome"]])

# Carregar os pontos de avaliação
def carregar_pontos_avaliacao(arquivo):
    pontos = []
    with open(arquivo, mode="r") as file:
        reader = csv.DictReader(file)
        for i, row in enumerate(reader):
            pontos.append({
                "id": i + 1,  # Adicionar ID único
                "nome": row["Nome"],
                "capacidade": int(row["Capacidade"]),
                "lat": float(row["Latitude"]),
                "lon": float(row["Longitude"]),
            })
    return pontos

# Carregar os candidatos
def carregar_candidatos(arquivo):
    candidatos = []
    with open(arquivo, mode="r") as file:
        reader = csv.DictReader(file)
        for i, row in enumerate(reader):
            candidatos.append({
                "id": i + 1,  # Adicionar ID único
                "nome": row["Name"],
                "lat": float(row["Latitude"]),
                "lon": float(row["Longitude"]),
            })
    return candidatos



In [2]:
# haversine
import math
from collections import defaultdict

def haversine(lat1, lon1, lat2, lon2):
    """
    Calcula a distância entre duas coordenadas de latitude e longitude usando a fórmula de Haversine.
    """
    R = 6371  # Raio da Terra em km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # Retorna a distância em km


In [15]:
def otimizacao_local(alocacoes, pontos_avaliacao, participantes, max_iteracoes=100):
    """
    Realiza a otimização local para reduzir o custo total da solução inicial.
    Interrompe após atingir o número máximo de iterações.

    Args:
        alocacoes (dict): Alocação inicial dos participantes.
        pontos_avaliacao (list): Lista de pontos de avaliação.
        participantes (list): Lista de participantes.
        max_iteracoes (int): Número máximo de iterações permitido.

    Returns:
        dict: Alocações otimizadas.
    """
    pontos_dict = {ponto["id"]: ponto for ponto in pontos_avaliacao}  # Para acesso rápido
    convergiu = False
    iteracoes = 0  # Contador de iterações

    while not convergiu and iteracoes < max_iteracoes:
        convergiu = True  # Assume que não haverá melhorias
        iteracoes += 1

        for ponto_id, participantes_ids in list(alocacoes.items()):
            ponto_atual = pontos_dict[ponto_id]

            for participante_id in participantes_ids[:]:  # Iterar sobre uma cópia para evitar problemas ao realocar
                participante = next(p for p in participantes if p["id"] == participante_id)
                menor_custo = haversine(participante["lat"], participante["lon"], ponto_atual["lat"], ponto_atual["lon"])
                melhor_ponto_id = ponto_id

                # Testar realocação para outros pontos de avaliação
                for outro_ponto in pontos_avaliacao:
                    if outro_ponto["id"] != ponto_id and len(alocacoes[outro_ponto["id"]]) < outro_ponto["capacidade"]:
                        novo_custo = haversine(participante["lat"], participante["lon"], outro_ponto["lat"], outro_ponto["lon"])
                        if novo_custo < menor_custo:  # Encontrou um ponto melhor
                            menor_custo = novo_custo
                            melhor_ponto_id = outro_ponto["id"]

                # Realocar se encontrou um ponto melhor
                if melhor_ponto_id != ponto_id:
                    alocacoes[ponto_id].remove(participante_id)
                    alocacoes[melhor_ponto_id].append(participante_id)
                    convergiu = False  # Houve uma melhoria, então continua otimizando

    print(f"Otimizador parou após {iteracoes} iterações.")
    return alocacoes


In [10]:
def alocar_participantes_ordenado(pontos_avaliacao, participantes):
    """
    Aloca os participantes aos pontos de avaliação de forma que cada participante seja alocado ao ponto mais próximo disponível.
    """
    alocacoes = defaultdict(list)

    for participante in participantes:
        # Calcular distâncias e ordenar os pontos de avaliação por proximidade
        distancias = sorted(
            [
                (haversine(participante["lat"], participante["lon"], ponto["lat"], ponto["lon"]), ponto["id"])
                for ponto in pontos_avaliacao
                   if len(alocacoes[ponto["id"]]) < ponto["capacidade"]
            ],
            key=lambda x: x[0]
        )

        # Seleciona o ponto de avaliação mais próximo com capacidade disponível
        if distancias:
            _, ponto_selecionado = distancias[0]  # O primeiro item é o mais próximo
            alocacoes[ponto_selecionado].append(participante["id"])

    return alocacoes


In [17]:
def alocar_participantes(pontos_avaliacao, participantes):
    # Inicializa um dicionário para rastrear quantos participantes estão alocados em cada ponto de avaliação
    alocacoes = defaultdict(list)

    for participante in participantes:
        distancias = []

        # Calcula a distância do participante a cada ponto de avaliação e verifica a capacidade disponível
        for ponto in pontos_avaliacao:
            if len(alocacoes[ponto["id"]]) < ponto["capacidade"]:
                distancia = haversine(participante["lat"], participante["lon"], ponto["lat"], ponto["lon"])
                distancias.append((distancia, ponto["id"]))

        # Seleciona o ponto de avaliação mais próximo com capacidade disponível
        if distancias:
            _, ponto_selecionado = min(distancias, key=lambda x: x[0])
            alocacoes[ponto_selecionado].append(participante["id"])

    return alocacoes


In [4]:
def calcular_custo_total(alocacoes, pontos_avaliacao, participantes):
    """
    Calcula o custo total como o somatório das distâncias percorridas por todos os participantes.
    """
    custo_total = 0

    # Criar um dicionário para acesso rápido aos dados dos pontos de avaliação
    pontos_dict = {ponto["id"]: ponto for ponto in pontos_avaliacao}

    # Iterar pelas alocações
    for ponto_id, participantes_ids in alocacoes.items():
        ponto = pontos_dict[ponto_id]  # Dados do ponto de avaliação

        for participante_id in participantes_ids:
            # Encontrar os dados do participante
            participante = next(p for p in participantes if p["id"] == participante_id)

            # Calcular a distância e somar ao custo total
            distancia = haversine(participante["lat"], participante["lon"], ponto["lat"], ponto["lon"])
            custo_total += distancia

    return custo_total


In [8]:
# Carregar dados
pontos_avaliacao = carregar_pontos_avaliacao("pontos-avaliacao.csv")
candidatos = carregar_candidatos("candidatos.csv")

In [11]:
# Alocar participantes
alocacoes_iniciais = alocar_participantes_ordenado(pontos_avaliacao, candidatos)


In [13]:
# Calcular o custo inicial
custo_inicial = calcular_custo_total(alocacoes_iniciais, pontos_avaliacao, candidatos)
print(f"Custo inicial: {custo_inicial:.2f} km")

Custo inicial: 218676.65 km


In [16]:
# Aplicar otimização local
alocacoes_otimizadas = otimizacao_local(alocacoes_iniciais, pontos_avaliacao, candidatos)


Otimizador parou após 1 iterações.


In [17]:
# Calcular o custo após a otimização
custo_otimizado = calcular_custo_total(alocacoes_otimizadas, pontos_avaliacao, candidatos)
print(f"Custo após otimização: {custo_otimizado:.2f} km")


Custo após otimização: 218676.65 km


In [18]:
# Verificar a melhoria
print(f"Melhoria no custo: {custo_inicial - custo_otimizado:.2f} km")

Melhoria no custo: 0.00 km


In [19]:


# Salvar as alocações no formato desejado
salvar_alocacoes_csv(alocacoes_iniciais, pontos_avaliacao, candidatos, "alocacoes1.csv")

print("Arquivo 'alocacoes.csv' gerado com as alocações dos participantes.")

Arquivo 'alocacoes.csv' gerado com as alocações dos participantes.


In [20]:
import pandas as pd


df = pd.read_csv("alocacoes1.csv")
df.head()
df.describe()

,Cod Participante
count,100000.000000
mean,50000.500000
std,28867.657797
min,1.000000
25%,25000.750000
50%,50000.500000
75%,75000.250000
max,100000.000000


### Gerador de dados

In [7]:
import random
import csv

# Função para gerar localizações aleatórias
def generate_random_location(lat_range, lon_range):
    lat = random.uniform(*lat_range)
    lon = random.uniform(*lon_range)
    return lat, lon

# Parâmetros da cidade de Rondonópolis
lat_range = (-16.5, -16.3)
lon_range = (-54.8, -54.5)

# Parâmetros para pontos de avaliação
num_candidates = 100000
capacity_per_point = 600  # 20 salas com 30 pessoas cada

# Calcular o número de pontos de avaliação necessários
num_points = num_candidates // capacity_per_point + (1 if num_candidates % capacity_per_point != 0 else 0)

# Geração dos pontos de avaliação
points_of_evaluation = []
for i in range(num_points):
    lat, lon = generate_random_location(lat_range, lon_range)
    points_of_evaluation.append({"nome": f"Ponto {i+1}", "capacity": capacity_per_point, "latitude": lat, "longitude": lon})

# Geração dos candidatos
candidates = []
for i in range(num_candidates):
    lat, lon = generate_random_location(lat_range, lon_range)
    candidates.append({"nome": f"Candidato {i+1}", "latitude": lat, "longitude": lon})

# Salvar os pontos de avaliação em um arquivo CSV
with open("./pontos-avaliacao.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Nome", "Capacidade", "Latitude", "Longitude"])
    for point in points_of_evaluation:
        writer.writerow([point["nome"], point["capacity"], point["latitude"], point["longitude"]])

# Salvar os candidatos em um arquivo CSV
with open("./candidatos.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Latitude", "Longitude"])
    for candidate in candidates:
        writer.writerow([candidate["nome"], candidate["latitude"], candidate["longitude"]])

print(f"Pontos de avaliação gerados: {num_points}")
print(f"Arquivo 'pontos-avaliacao.csv' gerado com {num_points} pontos de avaliação.")
print(f"Arquivo 'candidatos.csv' gerado com {num_candidates} candidatos.")

Pontos de avaliação gerados: 167
Arquivo 'pontos-avaliacao.csv' gerado com 167 pontos de avaliação.
Arquivo 'candidatos.csv' gerado com 100000 candidatos.
